In [ ]:
OPENAI_API_KEY = "xxxx"

from openai import OpenAI 
import os
import base64

MODEL="gpt-4o-mini"
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", OPENAI_API_KEY))

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def extract_text(base64_image):
    response = client.chat.completions.create(
        model = MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that can extract text from an image and return in a latex markdown format"},
            {"role": "user", "content": [
                {"type": "text", "text": "Can you extract all the text from this image?"},
                {"type": "image_url", "image_url": {
                    "url": f"data:image/png;base64,{base64_image}"}
                }
            ]}
        ],
        temperature=0.0,
    )

    return response.choices[0].message.content

def save_to_markdown(text, path):
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)
    
    print(f"saved to {path}")

In [3]:
from natsort import natsorted
from tqdm.auto import tqdm
import os

text = ''
FOLDER_PATH = r"E:\pi_school\eva-work-notes\text-extraction\data\s3\ground_truths\elsevier\two"
for image in natsorted(os.listdir(FOLDER_PATH)):
    base64_image = encode_image(os.path.join(FOLDER_PATH, image))
    text += extract_text(base64_image)

save_to_markdown(text, FOLDER_PATH + '.md')

saved to E:\pi_school\eva-work-notes\text-extraction\data\s3\ground_truths\elsevier\two.md
